In [195]:
import warnings
warnings.filterwarnings('ignore')

In [196]:
import pandas as pd
import numpy as np

In [197]:
data = pd.read_csv('botswana.tsv', delimiter='\t')
print(data.shape)

data.head()

(4361, 15)


,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [198]:
religions = data['religion'].values
unique_religions = len(list(dict.fromkeys(religions)))

unique_religions

4

In [199]:
drop_nan = data.dropna()
objects, _ = drop_nan.shape

objects

1834

In [200]:
data['nevermarr'] = [1 if data.loc[i, 'evermarr'] == 0 else 0 for i in range(data.shape[0])]
data.drop('evermarr', axis=1, inplace=True)

data.agefm[data.agefm.isnull()] = 0
data.heduc[data.heduc.isnull() & data.nevermarr.values == 1] = -1

In [201]:
data.heduc.isnull().value_counts()

False    4238
True      123
Name: heduc, dtype: int64

In [202]:
data['idlnchld_noans'] = 0
data.loc[data.idlnchld.isnull(), 'idlnchld_noans'] = 1

data['heduc_noans'] = 0
data.loc[data.heduc.isnull(), 'heduc_noans'] = 1

data['usemeth_noans'] = 0
data.loc[data.usemeth.isnull(), 'usemeth_noans'] = 1

In [203]:
data.idlnchld[data.idlnchld.isnull()] = -1
data.heduc[data.heduc.isnull()] = -2
data.usemeth[data.usemeth.isnull()] = -1

In [204]:
data.dropna(inplace=True)

rows, cols = data.shape
rows * cols

78264

In [205]:
import statsmodels.formula.api as smf

In [206]:
data.columns

Index(['ceb', 'age', 'educ', 'religion', 'idlnchld', 'knowmeth', 'usemeth',
       'agefm', 'heduc', 'urban', 'electric', 'radio', 'tv', 'bicycle',
       'nevermarr', 'idlnchld_noans', 'heduc_noans', 'usemeth_noans'],
      dtype='object')

In [207]:
formula = 'ceb ~ ' + ' + '.join(data.columns[1:])
formula

'ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + radio + tv + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans'

In [208]:
model_1 = smf.ols(formula, data=data)
fitted_1 = model_1.fit()
print(fitted_1.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Tue, 02 Jun 2020   Prob (F-statistic):               0.00
Time:                        15:48:07   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [209]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted_1.resid, fitted_1.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [210]:
model_2 = smf.ols(formula, data=data)
fitted_2 = model_2.fit(cov_type='HC1')
print(fitted_2.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     345.0
Date:                Tue, 02 Jun 2020   Prob (F-statistic):               0.00
Time:                        15:48:33   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [211]:
formula2 = 'ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + bicycle \
+ nevermarr + idlnchld_noans + heduc_noans + usemeth_noans'

model_3 = smf.ols(formula2, data)
fitted_3 = model_3.fit()
print(fitted_3.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     559.5
Date:                Tue, 02 Jun 2020   Prob (F-statistic):               0.00
Time:                        15:48:56   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.198     -5.

In [212]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted_3.resid, fitted_3.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [213]:
model_4 = smf.ols(formula2, data=data)
fitted_4 = model_4.fit(cov_type='HC1')
print(fitted_4.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Tue, 02 Jun 2020   Prob (F-statistic):               0.00
Time:                        15:49:25   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

In [219]:
print(f'p-value: {model_2.fit().compare_f_test(model_4.fit())[1]:.4f}')

p-value: 0.4672
